In [ ]:
!wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2 --no-check-certificate
!tar jxvf  LJSpeech-1.1.tar.bz2 > /dev/null 2>&1

In [ ]:
import sagemaker
# input_data_uri = sagemaker.session.Session().upload_data(path='LJSpeech-1.1/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='LJSpeech-1.1')
input_data_uri='s3://sagemaker-us-east-2-290000338583/LJSpeech-1.1'

In [ ]:
from sagemaker.pytorch import PyTorch

In [ ]:
estimator = PyTorch(
    entry_point='train.py',
    source_dir = './src/',
    py_version='py3', 
    framework_version='1.4.0',
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'config':'config_v1.json'
    }
)
estimator.fit(input_data_uri)